# NER Dataset builder

On va importer le fichier .csv qui va contenir les phrases d'entrainement de notre model qui ont été récoltés auprès de nos utilisateurs. C'est notre base à partir de laquelle on va pouvoir entrainer notre bot.

In [10]:
# Import training phrases csv file
import csv

csv_path = "/content/corpus.csv"

corpus = []

with open(csv_path, 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        corpus.append({'text': row[0], 'dep': row[1], 
                        'des': row[2]})

corpus.pop(0)
        
print(corpus)

[{'text': 'Je cherche un train depuis Paris pour Marseille', 'dep': 'Paris', 'des': 'Marseille'}, {'text': 'Je suis à la recherche d’un billet depuis Paris pour Lyon', 'dep': 'Paris', 'des': 'Lyon'}, {'text': 'Est ce qu’il y’a un train pour Paris depuis Strasbourg ?', 'dep': 'Strasbourg', 'des': 'Paris'}, {'text': 'Y’a-t-il un trajet depuis Reims pour Châlons-en-Champagne ?', 'dep': 'Reims', 'des': 'Châlons-en-Champagne'}, {'text': 'Un train pour Lyon à partir de Saint-Etienne', 'dep': 'Saint-Etienne', 'des': 'Lyon'}, {'text': 'Un trajet de Paris à Reims', 'dep': 'Paris', 'des': 'Reims'}, {'text': 'Un train de Rennes à Bordeaux', 'dep': 'Rennes', 'des': 'Bordeaux'}, {'text': 'Un train pour Reims depuis Strasbourg', 'dep': 'Strasbourg', 'des': 'Reims'}, {'text': 'Un trajet pour Lyon depuis Nîmes', 'dep': 'Nîmes', 'des': 'Lyon'}, {'text': 'Est ce qu’il y’a des trains de Gare de l’Est à Strasbourg ?', 'dep': 'Gare de l’Est', 'des': 'Strasbourg'}, {'text': 'Je veux partir à Lille depuis Ga

In [11]:
# Get sample of the corpus 
print(corpus[0])

{'text': 'Je cherche un train depuis Paris pour Marseille', 'dep': 'Paris', 'des': 'Marseille'}


# Training Sets
Les données pour entrainées un model de machine learning existent sous trois formes:

*   Les données d'entrainement
*   Les données de validation
*   Les données de test

Toutes ces données vont être strucutrées de la même manière, c'est à dire sous la forme d'une liste de données structurées dans laquelle chaque index va contenir une chaine de caractère qui va représenter un texte (la requête de l'utilisateur).

Le second composant de nos données d'entrainement va va être composé d'une liste d'entités à extraire présentes dans ce texte avec pour chacune d'elle la position de départ dans la string, la position de fin, ainsi que le label auquel cette entité doit être assosiées.

Durant la phase d'entrainement de notre model, ces annotations vont permettre au réseau de neurone (architecture de ML utilisé par spaCy) d'apprendre de ces données pour être capable d'identifier correctement les entitées sur lequelles on souhaite l'entrainer.

Voici la forme que doit prendre notre set d'entrainement:

`TRAIN_DATA = [ (TEXT AS A STRING, {“entities”: [(START, END, LABEL)]}) ]`

In [13]:
import re

TRAIN_DATA = []

# Get start and end position of the token inside the text value
def get_position(text, token):
  for match in re.finditer(token, text):
    return (match.start(), match.end()) 

# Get entity format (START, END, LABEL)
def get_entity(text, token, label):
  start, end = get_position(text, token)
  return (start, end, label)

for sentence in corpus:
  text = sentence["text"]
  get_entity(text, sentence["des"], "DES")
  get_entity(text, sentence["dep"], "DEP")
  # item => (TEXT AS A STRING, {“entities”: [(START, END, LABEL)]}) 
  item = (text, {"entities": [get_entity(text, sentence["dep"], "DEP"), get_entity(text, sentence["des"], "DES")]})
  TRAIN_DATA.append(item)

print(TRAIN_DATA)

[('Je cherche un train depuis Paris pour Marseille', {'entities': [(27, 32, 'DEP'), (38, 47, 'DES')]}), ('Je suis à la recherche d’un billet depuis Paris pour Lyon', {'entities': [(42, 47, 'DEP'), (53, 57, 'DES')]}), ('Est ce qu’il y’a un train pour Paris depuis Strasbourg ?', {'entities': [(44, 54, 'DEP'), (31, 36, 'DES')]}), ('Y’a-t-il un trajet depuis Reims pour Châlons-en-Champagne ?', {'entities': [(26, 31, 'DEP'), (37, 57, 'DES')]}), ('Un train pour Lyon à partir de Saint-Etienne', {'entities': [(31, 44, 'DEP'), (14, 18, 'DES')]}), ('Un trajet de Paris à Reims', {'entities': [(13, 18, 'DEP'), (21, 26, 'DES')]}), ('Un train de Rennes à Bordeaux', {'entities': [(12, 18, 'DEP'), (21, 29, 'DES')]}), ('Un train pour Reims depuis Strasbourg', {'entities': [(27, 37, 'DEP'), (14, 19, 'DES')]}), ('Un trajet pour Lyon depuis Nîmes', {'entities': [(27, 32, 'DEP'), (15, 19, 'DES')]}), ('Est ce qu’il y’a des trains de Gare de l’Est à Strasbourg ?', {'entities': [(31, 44, 'DEP'), (47, 57, 'DES

In [14]:
# Get sample of training set
print(TRAIN_DATA[0])

('Je cherche un train depuis Paris pour Marseille', {'entities': [(27, 32, 'DEP'), (38, 47, 'DES')]})


# Conception du model NER

In [18]:
from pathlib import Path

model = None
output_dir=Path("/content")
n_iter=100

In [24]:
import spacy

#load the model
if model is not None:
    nlp = spacy.load(model)  
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.blank('fr')  
    print("Created blank 'fr' model")

#set up the pipeline

# create the built-in pipeline components and add them to the pipeline

# nlp.create_pipe works for built-ins that are registered with spaCy
if "ner" not in nlp.pipe_names:
  ner = nlp.create_pipe("ner")
  nlp.add_pipe(ner, last=True)
# otherwise, get it so we can add labels
else:
  ner = nlp.get_pipe("ner")
  
# add labels
for _, annotations in TRAIN_DATA:
  for ent in annotations.get("entities"):
    ner.add_label(ent[2])

# if 'ner' not in nlp.pipe_names:
#     ner = nlp.create_pipe('ner')
#     nlp.add_pipe(ner, last=True)
# else:
#     ner = nlp.get_pipe('ner')

Created blank 'fr' model


In [29]:
from spacy.util import minibatch, compounding

# get names of other pipes to disable them during training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
with nlp.disable_pipes(*other_pipes):  # only train NER
  # reset and initialize the weights randomly – but only if we're
  # training a new model
  if model is None:
    nlp.begin_training()
    for itn in range(n_iter):
      random.shuffle(TRAIN_DATA)
      losses = {}
      # batch up the examples using spaCy's minibatch
      batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
      for batch in batches:
        texts, annotations = zip(*batch)
        nlp.update(
          texts,  # batch of texts
          annotations,  # batch of annotations
          drop=0.5,  # dropout - make it harder to memorise data
          losses=losses)
        print("Losses", losses)

Losses {'ner': 42.30000400543213}
Losses {'ner': 81.7354245185852}
Losses {'ner': 109.14258527755737}
Losses {'ner': 137.9289677143097}
Losses {'ner': 165.7089648246765}
Losses {'ner': 192.3230414390564}
Losses {'ner': 197.84436774253845}
Losses {'ner': 23.723423719406128}
Losses {'ner': 41.88542175292969}
Losses {'ner': 67.98163390159607}
Losses {'ner': 85.37216711044312}
Losses {'ner': 105.21218621730804}
Losses {'ner': 124.38674831390381}
Losses {'ner': 127.79651390016079}
Losses {'ner': 13.146721318364143}
Losses {'ner': 24.966309919953346}
Losses {'ner': 38.954409793019295}
Losses {'ner': 52.78617876768112}
Losses {'ner': 67.20359390974045}
Losses {'ner': 85.5989980045706}
Losses {'ner': 88.91909411270171}
Losses {'ner': 19.60433554649353}
Losses {'ner': 34.36550099775195}
Losses {'ner': 48.87614529393613}
Losses {'ner': 61.93785675987601}
Losses {'ner': 74.55114651098847}
Losses {'ner': 87.02229211106896}
Losses {'ner': 90.75246684253216}
Losses {'ner': 12.148019835352898}
Losses

# Tester le model

In [30]:
 # test the trained model

 # IOB code of named entity tag. 3 means the token begins an entity, 2 means it
 # is outside an entity, 1 means it is inside an entity, and 0 means no entity 
 # tag is set.
 
for text, _ in TRAIN_DATA:
  doc = nlp(text)
  print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
  print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])

Entities [('Lyon', 'DES'), ('Saint-Etienne', 'DEP')]
Tokens [('Un', '', 2), ('train', '', 2), ('pour', '', 2), ('Lyon', 'DES', 3), ('à', '', 2), ('partir', '', 2), ('de', '', 2), ('Saint-Etienne', 'DEP', 3)]
Entities [('Nice', 'DEP'), ('Lyon', 'DES')]
Tokens [('Je', '', 2), ('recherche', '', 2), ('un', '', 2), ('train', '', 2), ('de', '', 2), ('Nice', 'DEP', 3), ('à', '', 2), ('Lyon', 'DES', 3)]
Entities [('Marseille', 'DES'), ('Grenoble', 'DEP')]
Tokens [('J’', '', 2), ('ai', '', 2), ('besoin', '', 2), ('de', '', 2), ('me', '', 2), ('rendre', '', 2), ('à', '', 2), ('Marseille', 'DES', 3), ('en', '', 2), ('partant', '', 2), ('de', '', 2), ('Grenoble', 'DEP', 3)]
Entities [('Paris', 'DEP'), ('Lyon', 'DES')]
Tokens [('Je', '', 2), ('suis', '', 2), ('à', '', 2), ('la', '', 2), ('recherche', '', 2), ('d’', '', 2), ('un', '', 2), ('billet', '', 2), ('depuis', '', 2), ('Paris', 'DEP', 3), ('pour', '', 2), ('Lyon', 'DES', 3)]
Entities [('Rennes', 'DEP'), ('Bordeaux', 'DES')]
Tokens [('Un', ''

In [31]:
for text, _ in TRAIN_DATA:
    doc = nlp(text)
    print('Entities', [(ent.text, ent.label_) for ent in doc.ents])

Entities [('Lyon', 'DES'), ('Saint-Etienne', 'DEP')]
Entities [('Nice', 'DEP'), ('Lyon', 'DES')]
Entities [('Marseille', 'DES'), ('Grenoble', 'DEP')]
Entities [('Paris', 'DEP'), ('Lyon', 'DES')]
Entities [('Rennes', 'DEP'), ('Bordeaux', 'DES')]
Entities [('Orléans', 'DEP'), ('Rouen', 'DES')]
Entities [('Lille', 'DES'), ('Gare du Nord', 'DEP')]
Entities [('Troyes', 'DEP'), ('Reims', 'DES')]
Entities [('Paris', 'DES'), ('Strasbourg', 'DEP')]
Entities [('Bordeaux', 'DEP'), ('Biarritz', 'DES')]
Entities [('Nice', 'DEP'), ('Fréjus', 'DES')]
Entities [('Reims', 'DEP'), ('Paris', 'DES')]
Entities [('Paris', 'DES'), ('Nancy', 'DEP')]
Entities [('Clermont-Ferrand', 'DES'), ('Vichy', 'DEP')]
Entities [('Paris', 'DEP'), ('Reims', 'DES')]
Entities [('Reims', 'DEP'), ('Châlons-en-Champagne', 'DES')]
Entities [('Lyon', 'DES'), ('Nîmes', 'DEP')]
Entities [('Paris', 'DEP'), ('Lyon', 'DES')]
Entities [('Gare de l’Est', 'DEP'), ('Strasbourg', 'DES')]
Entities [('Paris', 'DEP'), ('Marseille', 'DES')]
Ent